In [ ]:
"""
Purpose: Find clusters and their outliers, output the results to np arrays.
"""

import ast
import random
import numpy as np
np.set_printoptions(threshold='nan')
import scipy as sp
from scipy import stats
import pyfits
import math
import matplotlib.pyplot as plt
import heapq
import scipy.signal
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from numpy.random import RandomState
from multiprocessing import Pool
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import RadioButtons


identifier = raw_input('Enter the identifier of the data: ')

# nclusters could (someday *should*) be obtained through the optimalK.py script 
nclusters = int(raw_input('Enter the number of clusters expected: '))

# byFeature is an array organized by feature. [[all feature 1 data],[all feature 2 data],...] 
# Useful for plotting and finding extrema.
def byFeature(X):
    return [[X[i][j] for i in range(len(X))] for j in range(len(X[0]))]

def bounding_box(X):
    # X is the data that comes in, it's organized by lightcurve[[all features for lc 1],[features for lc2],...]
    # To find minima we need to consider all points for each feature seperate from the other features.
    Xbyfeature = byFeature(X)

    # xmin/xmax will be an array of the minimum/maximum values of the features
    xmin=[]
    xmax=[]
    # Create the minimum vertex, and the maximum vertex for the box
    for feature in range(60):
        xmin.append(min(Xbyfeature[feature]))
        xmax.append(max(Xbyfeature[feature]))
        
    return (xmin,xmax)

def KMeans_clusters(data,nclusters):
    # Run KMeans, get clusters
    npdata = np.array(data)
    est = KMeans(n_clusters=nclusters)
    est.fit(npdata)
    clusters = est.labels_
    centers = est.cluster_centers_
    return clusters, centers

def seperatedClusters(data,nclusters,clusters):
    """
    Args:
    data - all data, organized by lightcurve 
    nclusters - number of clusters
    clusters - cluster labels
    
    Purpose: Create arrays dataByCluster and clusterIndexes containing data seperated by
    by cluster.
    """
    dataByCluster = []
    clusterIndexes = []
    
    # Will try to stick to the following:
    # cluster i, lightcurve j, feature k
    for i in range(nclusters):
        # Keeping track of which points get pulled into each cluster:
        clusterIndexes.append([j for j in range(len(data)) if clusters[j]==i])
        # Separating the clusters out into their own arrays (w/in the cluster array)
        dataByCluster.append([data[clusterIndexes[i][j]] for j in range(len(clusterIndexes[i]))])
        
        #Alternatively: (might have had some issues with the above, not sure...)
        #dataByCluster.append([])
        #for j in range(len(clusterIndexes[i])):
        #    dataByCluster[i].append(data[clusterIndexes[i][j]])
    return dataByCluster, clusterIndexes

def distances(pointsForDistance,referencePoint):
    """
    Args:
    cluster - array of any number of points, each point an array of it's features
    centerloc - array of the feautres for a single point
    
    Purpose: 
    This will calculate the distances of a group of points to a given point.
    """
    distFromCenter = [0 for j in range(len(pointsForDistance))] # reinitializing for each cluster
    # loop for each lightcurve of the cluster
    for j in range(len(pointsForDistance)):
        dataloc=pointsForDistance[j] # coordinates of the datapoint
        sqrd=0
        # loop for each feature of the lightcurve 
        for k in range(len(referencePoint)):
            sqrd+=pow(dataloc[k]-centerloc[k],2) # (x-x0)^2+(y-y0)^2+...
        distance = pow(sqrd,0.5) # sqrt((x-x0)^2+(y-y0)^2+...)
        distFromCenter[j]=distance 
    return distFromCenter

def beyondCutoff(cutoff,distances):
    """
    returns: outliers and typical arrays
        outlier array contains indexes of the specific cluster array that are beyond the cutoff
        typical array contains single index of the cluster array that is nearest the center
    
    Args:
    cutoff - a number, everything beyond this number is considered an outlier
    distances - array containing distances to each lightcurve point from the center for a given cluster
    """
    outliers=[j for j in range(len(distances)) if distances[j]>=cutoff] # recalculated for each cluster
    typical=[j for j in range(len(distances)) if distances[j]==min(distances)]
    return outliers,typical

def outliers(data,clusters,centers,files):
    """
    Args:
    data - all the data
    clusters - the cluster labels from kmeans. DBSCAN will likely require different methodology
    centers - locations of the cluster centers
    
    Purpose:
    Separate out the data on the edge of the clusters which are the most likely anomalous data.
    
    """
    
    nclusters = len(centers)

    # Sanity check, if this isn't an outlier than something is wrong.
    """controlPoint = np.array([10000 for i in range(len(data[0]))])
    
    data=np.append(data,[controlPoint],axis=0)
    clusters=np.append(clusters,[1],axis=0)"""
    
    """
    Initializing arrays
    """
    
    cluster, clusterIndexes = seperatedClusters(data,nclusters,clusters)
    twoSigma = [] #probably doesn't need an array 
    distFromCenter = []
    allTypical=[]
    allOutliers=[]
    # Will try to stick to the following:
    # cluster i, lightcurve j, feature k
    for i in range(nclusters):
                
        """
        ========== Checking density of clusters ============
        Given that some of the clusters may have 0 spread in a given feature, I'm not confident that this is meaningful.
        """
        (xmin,xmax)=bounding_box(cluster[i]) # get the minimum and maximum values for both
        diff=[xmax[n]-xmin[n] for n in range(len(xmin))]
        volOfClusterBB=1.0
        for n in diff:
            if n!=0:
                volOfClusterBB*=n
        densOfCluster=len(cluster[i])/volOfClusterBB
        """print("Cluster: %s, Density: %s" %(i,densOfCluster))
        print volOfClusterBB"""
        
        """
        ========== Finding points outside of the cutoff ===========
        """
        # not currently using this cutoff.
        twoSigma.append(2*np.std(cluster[i]))
        
        """
            ==== Calculating distances to each point ====
        """
        # Calculate distances to each point in each cluster to the center of its cluster
        distFromCenter.append(distances(cluster[i],centers[i]))
        
        """
            ==== Finding outliers and the standard (defined by the closest to the center) ====
        """
        cutoff=.5*max(distFromCenter[i])
        
        outliers,typical = beyondCutoff(cutoff,distFromCenter[i])
        
        # Arbitrarily large integer to indicate a new cluster is being considered.
        # will need made bigger if considering 1 billion+ lightcurves
        allTypical.append(987654321)
        # Add typical lightcurve to list. Only 1 produced at present, but this may change. The following
        # accounts for adding more typicals later.
        for j in typical:    
            allTypical.append(clusterIndexes[i][j])
            
        # place outliers from this cluster into general outlier list        
        allOutliers.append(987654321)
        if len(outliers)==0:
            print("none")
        else:
            for j in outliers:
                allOutliers.append(clusterIndexes[i][j])
    
    ncluster=1            
    print("Typical")
    for i in allTypical:
        if i == 987654321:
            print('')
            print("Cluster: %s"%cluster)
            cluster+=1
        else:
            print files[i]
            outputfile = open('c%soutlierfilelist'%(cluster-1),'w')
            outputfile.write('%s\n'%files[i])
    print("Outliers")
    ncluster=1
    for i in allOutliers:
        if i == 987654321:
            print('')
            print("Cluster: %s"%cluster)
            cluster+=1
        else:
            print files[i]
            with open('c%soutlierfilelist'%(cluster-1),'a') as outputfile:
                outputfile.write('%s\n'%files[i])
        
    return allOutliers,allTypical



"""ffeaturesID = str(identifier)+'dataByFeature.npy'
ffeatures = np.load(ffeaturesID)"""
dataID = str(identifier)+'dataByLightCurve.npy'
data = np.load(dataID)
ffeatures=[[data[i][j] for i in range(len(data))] for j in range(len(data[0]))]
filelist = str(identifier)+'filelist'
files = [line.strip() for line in open(filelist)]

clusterlabels,outliers,typical=KMeans_clusters(data,nclusters)

dataByCluster,clusterIndexes = seperatedClusters(data,nclusters,clusters)

for i in range(nclusters):
    for j in clusterIndexes[i]:
        outputfile = open('c%soutlierfilelist'%(ncluster+1),'w')
        outputfile.write('%s\n'%files[j])

for i in outliers:
    if i == 987654321:
        print('')
        print("Cluster: %s"%cluster)
        cluster+=1
    else:
        print files[i]
        with open('c%soutlierfilelist'%(cluster-1),'a') as outputfile:
            outputfile.write('%s\n'%files[i])

In [3]:
""" 
This will plot individual clusters into a 3D scatter and on choosing (clicking on) a point, 
plot the point's associated lightcurve. The 3 axes will be selectable via radiobuttons.

This may be applied to any scatter of data with associated lightcurves (assuming the 
lightcurve data is available and the filelist appropriately named).

There are two necessary files that need to be generated beforehand: 
the file list containing the lightcurve identifiers and a numpy array file containing 
the calculated features of each lightcurve. Theoretically it should not matter how the 
features were calculated, or how many features there are.

Example file list:

Name: exfilelist
Content:
lightcurve1_llc.fits
lightcurve2_llc.fits
lightcurve3_llc.fits

Note: There can be no blank lines.

Example numpy file:
Name: exdataByLightCurve.npy
Content (when read-in by np.load()):
[[data for lightcurve1]
 [data for lightcurve2]
 [data for lightcurve3]]
 
Note: The filelist and the .npy must have the above fromat. 
filelist: identifier+'filelist'    npy: identifier+'dataByLighCurve.npy'
In the example above the identifier was 'ex'.
"""

import ast
import random
import numpy as np
np.set_printoptions(threshold='nan')
import scipy as sp
from scipy import stats
import pyfits
import math
import pylab as pl
import matplotlib.pyplot as plt
import heapq
from operator import xor
import scipy.signal
from numpy import float64
#import astroML.time_series
#import astroML_addons.periodogram
#import cython
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from numpy.random import RandomState
#rng = RandomState(42)
import itertools
import commands
# import utils
import itertools
#from astropy.io import fits
from multiprocessing import Pool
from mpl_toolkits.mplot3d import Axes3D,proj3d
from matplotlib.widgets import RadioButtons

# reorganizeArray applied to a set of points returns an array organized by feature. [[all feature 1 data],[all feature 2 data],...] 
# Useful for plotting and finding extrema.
# Applied to data organized by feature will return an array organized by point. [[point 1 feature 1,point 1 feature 2, ...],[point 2 feature 1, point 2 feature 2,...]]
# Useful fro analyzing points of data (ex. finding closest points to a mouseclick)

def reorganizeArray(X): # This is just a transposition, could be accomplished by making X an np.array then transposing via X.T
    return [[X[i][j] for i in range(len(X))] for j in range(len(X[0]))]

"""--- Importing the relevant data ---"""
identifier = raw_input('Enter the identifier (including cluster #) of the data: ')
dataID = str(identifier)+'dataByLightCurve.npy'
data = np.load(dataID)
ffeatures=reorganizeArray(data)
filelist = str(identifier)+'filelist'
files = [line.strip() for line in open(filelist)]
# I will attempt to use the format c+Cluster#+QuarterID
# For example cluster #4 from TTQ8 would be c4TTQ8
# This will be used to read in the relevant filelist and the numpy array which
# should be created by the Kmeans3D (or DBSCAN in the future) python script.



def read_revant_pars(parfile):
    pars = [line.strip() for line in open(parfile)]
    pararr = np.zeros((len(pars), 60))
    for i in range(len(pars)):
        pararr[i] = np.fromstring(pars[i], dtype=float, sep=' ')
    return pararr

def read_kepler_curve(file):
    lc = pyfits.getdata(file)
    t = lc.field('TIME')
    f = lc.field('PDCSAP_FLUX')
    err = lc.field('PDCSAP_FLUX_ERR')
    nf = f / np.median(f)
 
    nf = nf[np.isfinite(t)]
    t = t[np.isfinite(t)]
    t = t[np.isfinite(nf)]
    nf = nf[np.isfinite(nf)]

    return t, nf, err

def dataScatter(data,lightcurveData):
    
    """--- Organizing data and Labels ---"""
    
    ffeatures = reorganizeArray(data)
    # Set the dictionaries that contain labels and data. Radiobuttons will have labels from listoffeatures
    # Betterlabels will contain the titles of axes we'll actually want on there
    listoffeatures = ['longtermtrend', 'meanmedrat', 'skews', 'varss', 'coeffvar', 'stds', 'numoutliers', 'numnegoutliers', 'numposoutliers', 'numout1s', 'kurt', 'mad', 'maxslope', 'minslope', 'meanpslope', 'meannslope', 'g_asymm', 'rough_g_asymm', 'diff_asymm', 'skewslope', 'varabsslope', 'varslope', 'meanabsslope', 'absmeansecder', 'num_pspikes', 'num_nspikes', 'num_psdspikes', 'num_nsdspikes','stdratio', 'pstrend', 'num_zcross', 'num_pm', 'len_nmax', 'len_nmin', 'mautocorrcoef', 'ptpslopes', 'periodicity', 'periodicityr', 'naiveperiod', 'maxvars', 'maxvarsr', 'oeratio', 'amp', 'normamp','mbp', 'mid20', 'mid35', 'mid50', 'mid65', 'mid80', 'percentamp', 'magratio', 'sautocorrcoef', 'autocorrcoef', 'flatmean', 'tflatmean', 'roundmean', 'troundmean', 'roundrat', 'flatrat']
    
    betterlabels = ['longtermtrend', 'meanmedrat', 'skews', 'varss', 'coeffvar', 'stds', 'numoutliers', 'numnegoutliers', 'numposoutliers', 'numout1s', 'kurt', 'mad', 'maxslope', 'minslope', 'meanpslope', 'meannslope', 'g_asymm', 'rough_g_asymm', 'diff_asymm', 'skewslope', 'varabsslope', 'varslope', 'meanabsslope', 'absmeansecder', 'num_pspikes', 'Number of Negative Spikes (Slope > 3*sigma)', 'num_psdspikes', 'num_nsdspikes','stdratio', 'pstrend', 'Number of Longterm Trendline Crossings', 'Number of Peaks', 'len_nmax', 'len_nmin', 'mautocorrcoef', 'ptpslopes', 'periodicity', 'periodicityr', 'naiveperiod', 'maxvars', 'maxvarsr', 'oeratio', 'amp', 'normamp','mbp', 'mid20', 'mid35', 'mid50', 'mid65', 'mid80', 'percentamp', 'magratio', 'sautocorrcoef', 'autocorrcoef', 'flatmean', 'tflatmean', 'roundmean', 'troundmean', 'roundrat', 'flatrat']
    
    # labeldict connects the variable's code name in listoffeatures to it's presentable name in betterlabels
    
    labeldict= dict(zip(listoffeatures,betterlabels))
    # datadict connects label to data
    datadict = dict(zip(listoffeatures,ffeatures))
    
    # labellist keeps track of what each axis should be labelled
    # initialized with some interesting axes (based on past experience)
    labellist=[labeldict[listoffeatures[30]],labeldict[listoffeatures[31]],labeldict[listoffeatures[25]]]
    
    # axesdict could probably be accomoplished with a list, see the new list currentData,
    # I'm partial to the dictionary because it helps me keep things associated directly.
    
    axesdict = {'xaxis':ffeatures[30],'yaxis':ffeatures[31],'zaxis':ffeatures[25]}
    currentData = [axesdict['xaxis'],axesdict['yaxis'],axesdict['zaxis']]
    
    """--- Plot initializing stuff ---"""
    fig = plt.figure(1)
    ax = fig.add_subplot(211, projection='3d')
    currentData = [axesdict['xaxis'],axesdict['yaxis'],axesdict['zaxis']]
    ax.scatter(axesdict['xaxis'], axesdict['yaxis'], axesdict['zaxis'])
    plt.subplots_adjust(left=0.3)
    ax.set_xlabel(labellist[0])
    ax.set_ylabel(labellist[1])
    ax.set_zlabel(labellist[2])
    
    # empty subplot for lightcurves
    ax2 = fig.add_subplot(212)

    """
    Matplotlib radiobuttons format poorly when there are more than ~5 buttons, this is an issue that will be resolved at some point in the future.
    For the time being, I'm making a bunch of radiobutton sections with 5 buttons apeice. Each column represents a differnt axis.
    The most recently clicked button will be the active one, so the color coding is less than useful.

    It's not pretty but it works.
    """
    
    """--- Radiobuttons ---"""
    # Creating dimensions for 12 boxes w/ 5 buttons each.
    rax1p = [[0.0,.92-i/12.0,0.1,1.0/12.0] for i in range(12)]
    # Making the boxes
    rax10,rax11,rax12 = plt.axes(rax1p[0]), plt.axes(rax1p[1]), plt.axes(rax1p[2]) 
    rax13,rax14,rax15 = plt.axes(rax1p[3]), plt.axes(rax1p[4]), plt.axes(rax1p[5])
    rax16,rax17,rax18 = plt.axes(rax1p[6]), plt.axes(rax1p[7]), plt.axes(rax1p[8])
    rax19,rax110,rax111 = plt.axes(rax1p[9]), plt.axes(rax1p[10]), plt.axes(rax1p[11])
    # Populating the boxes
    radio10,radio11 = RadioButtons(rax10, listoffeatures[0:5]),RadioButtons(rax11, listoffeatures[5:10])
    radio12,radio13 = RadioButtons(rax12, listoffeatures[10:15]),RadioButtons(rax13, listoffeatures[15:20])
    radio14,radio15 = RadioButtons(rax14, listoffeatures[20:25]),RadioButtons(rax15, listoffeatures[25:30])
    radio16,radio17 = RadioButtons(rax16, listoffeatures[30:35]),RadioButtons(rax17, listoffeatures[35:40])
    radio18,radio19 = RadioButtons(rax18, listoffeatures[40:45]),RadioButtons(rax19, listoffeatures[45:50])
    radio110,radio111 = RadioButtons(rax110, listoffeatures[50:55]),RadioButtons(rax111, listoffeatures[55:])
    
    # What happens on clicking a radiobutton (changes data for the given axis)
    def axis1(label):
        ax.cla()
        axesdict['xaxis'] = [datadict[label]]
        currentData = [axesdict['xaxis'],axesdict['yaxis'],axesdict['zaxis']]
        ax.scatter(currentData[0], currentData[1], currentData[2])
        labellist[0] = labeldict[label]
        ax.set_xlabel(labellist[0])
        ax.set_ylabel(labellist[1])
        ax.set_zlabel(labellist[2])
        plt.draw()
    
    # linking buttons to action
    radio10.on_clicked(axis1)
    radio11.on_clicked(axis1)
    radio12.on_clicked(axis1)
    radio13.on_clicked(axis1)
    radio14.on_clicked(axis1)
    radio15.on_clicked(axis1)
    radio16.on_clicked(axis1)
    radio17.on_clicked(axis1)
    radio18.on_clicked(axis1)
    radio19.on_clicked(axis1)
    radio110.on_clicked(axis1)
    radio111.on_clicked(axis1)


    rax2p = [[0.1,.92-i/12.0,0.1,1.0/12.0] for i in range(12)]

    rax20,rax21,rax22,rax23,rax24,rax25,rax26,rax27,rax28,rax29,rax210,rax211 = plt.axes(rax2p[0]), plt.axes(rax2p[1]), plt.axes(rax2p[2]), plt.axes(rax2p[3]), plt.axes(rax2p[4]), plt.axes(rax2p[5]), plt.axes(rax2p[6]), plt.axes(rax2p[7]), plt.axes(rax2p[8]), plt.axes(rax2p[9]), plt.axes(rax2p[10]), plt.axes(rax2p[11])

    radio20, radio21, radio22, radio23, radio24, radio25, radio26, radio27, radio28, radio29, radio210, radio211 = RadioButtons(rax20, listoffeatures[0:5]), RadioButtons(rax21, listoffeatures[5:10]), RadioButtons(rax22, listoffeatures[10:15]), RadioButtons(rax23, listoffeatures[15:20]), RadioButtons(rax24, listoffeatures[20:25]), RadioButtons(rax25, listoffeatures[25:30]), RadioButtons(rax26, listoffeatures[30:35]), RadioButtons(rax27, listoffeatures[35:40]), RadioButtons(rax28, listoffeatures[40:45]), RadioButtons(rax29, listoffeatures[45:50]), RadioButtons(rax210, listoffeatures[50:55]), RadioButtons(rax211, listoffeatures[55:])
    
    def axis2(label):
        ax.cla()
        axesdict['yaxis'] = [datadict[label]]
        currentData = [axesdict['xaxis'],axesdict['yaxis'],axesdict['zaxis']]
        ax.scatter(currentData[0], currentData[1], currentData[2])
        labellist[1] = label
        ax.set_xlabel(labeldict[labellist[0]])
        ax.set_ylabel(labeldict[labellist[1]])
        ax.set_zlabel(labeldict[labellist[2]])
        plt.draw()

    radio20.on_clicked(axis2)
    radio21.on_clicked(axis2)
    radio22.on_clicked(axis2)
    radio23.on_clicked(axis2)
    radio24.on_clicked(axis2)
    radio25.on_clicked(axis2)
    radio26.on_clicked(axis2)
    radio27.on_clicked(axis2)
    radio28.on_clicked(axis2)
    radio29.on_clicked(axis2)
    radio210.on_clicked(axis2)
    radio211.on_clicked(axis2)

    rax3p = [[0.2,.92-i/12.0,0.1,1.0/12.0] for i in range(12)]

    rax30,rax31,rax32,rax33,rax34,rax35,rax36,rax37,rax38,rax39,rax310,rax311 = plt.axes(rax3p[0]), plt.axes(rax3p[1]), plt.axes(rax3p[2]), plt.axes(rax3p[3]), plt.axes(rax3p[4]), plt.axes(rax3p[5]), plt.axes(rax3p[6]), plt.axes(rax3p[7]), plt.axes(rax3p[8]), plt.axes(rax3p[9]), plt.axes(rax3p[10]), plt.axes(rax3p[11])

    radio30, radio31, radio32, radio33, radio34, radio35, radio36, radio37, radio38, radio39, radio310, radio311 = RadioButtons(rax30, listoffeatures[0:5]), RadioButtons(rax31, listoffeatures[5:10]), RadioButtons(rax32, listoffeatures[10:15]), RadioButtons(rax33, listoffeatures[15:20]), RadioButtons(rax34, listoffeatures[20:25]), RadioButtons(rax35, listoffeatures[25:30]), RadioButtons(rax36, listoffeatures[30:35]), RadioButtons(rax37, listoffeatures[35:40]), RadioButtons(rax38, listoffeatures[40:45]), RadioButtons(rax39, listoffeatures[45:50]), RadioButtons(rax310, listoffeatures[50:55]), RadioButtons(rax311, listoffeatures[55:])

    def axis3(label):
        ax.cla()
        axesdict['zaxis'] = [datadict[label]]
        currentData = [axesdict['xaxis'],axesdict['yaxis'],axesdict['zaxis']]
        ax.scatter(currentData[0], currentData[1], currentData[2])
        labellist[2] = label
        ax.set_xlabel(labeldict[labellist[0]])
        ax.set_ylabel(labeldict[labellist[1]])
        ax.set_zlabel(labeldict[labellist[2]])
        plt.draw()
        
    radio30.on_clicked(axis3)
    radio31.on_clicked(axis3)
    radio32.on_clicked(axis3)
    radio33.on_clicked(axis3)
    radio34.on_clicked(axis3)
    radio35.on_clicked(axis3)
    radio36.on_clicked(axis3)
    radio37.on_clicked(axis3)
    radio38.on_clicked(axis3)
    radio39.on_clicked(axis3)
    radio310.on_clicked(axis3)
    radio311.on_clicked(axis3)
    
    """--- End Radiobuttons ---"""
    
    def distance(point, event):
        """Return distance between mouse position and given data point

        Args:
            point (np.array): np.array of shape (3,), with x,y,z in data coords
            event (MouseEvent): mouse event (which contains mouse position in .x and .xdata)
        Returns:
            distance (np.float64): distance (in screen coords) between mouse pos and data point
        """
        assert point.shape == (3,), "distance: point.shape is wrong: %s, must be (3,)" % point.shape

        # Project 3d data space to 2d data space
        x2, y2, _ = proj3d.proj_transform(point[0], point[1], point[2], ax.get_proj())
        # Convert 2d data space to 2d screen space
        x3, y3 = ax.transData.transform((x2, y2))

        return np.sqrt ((x3 - event.x)**2 + (y3 - event.y)**2)
    
    def calcClosestDatapoint(X, event):
        """"Calculate which data point is closest to the mouse position.
        
        Args:
            X (np.array) - array of points, of shape (numPoints, 3)
            event (MouseEvent) - mouse event (containing mouse position)
        Returns:
            smallestIndex (int) - the index (into the array of points X) of the element closest to the mouse position
        """
        distances = [distance (X[i, 0:3], event) for i in range(X.shape[0])]
        return np.argmin(distances)
    
    def drawData(X, index):
        # data needs to be in format of points w/ just the 3 relevant dimensions.
        # the data, therefore, needs redefined every time a new feature is chosen.
        x = X[0][index] 
        y = X[1][index]
        ax2.cla()
        ax2.scatter(x,y)
        plt.draw()
        
    """
    IN PROGRESS: WANT TO ANNOTATE CENTER OF CLUSTER
    def annotateCenter(X,index):
        x2, y2, _ = proj3d.proj_transform(X[index, 0], X[index, 1], X[index, 2], ax.get_proj())
        annotatePlot.label = plt.annotate( "Center",
            xy = (x2, y2), xytext = (-20, 20), textcoords = 'offset points', ha = 'right', va = 'bottom',
            bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
            arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
        fig.canvas.draw()
        """

    def onMouseClick(event):
        """Event that is triggered when mouse is clicked. Shows lightcurve for data point closest to mouse."""
        currentData1 = np.array(reorganizeArray(currentData))
        closestIndex = calcClosestDatapoint(currentData1, event)
        drawData(lightcurveData, closestIndex)
        #annotateCenter(lightcurveData,centerIndex)
    
    
    fig.canvas.mpl_connect('button_press_event', onMouseClick)  # on mouse click
    plt.show()

if __name__ == '__main__':
    tArr = []
    nfArr = []
    for f in files:
        t,nf,err=read_kepler_curve(f) #t, nf, err arrays of each for the specific lc
        tArr.append(t)
        nfArr.append(nf)
    # tArr and nfArr both arrays containing arrays. tArr[0] contains the array for time of the first lc
    lightcurveData = np.array([tArr,nfArr]) # Array of the 2 arrays, lightcurveData[0][0] is tArr[0]
    dataScatter(data,lightcurveData)

Enter the identifier (including cluster #) of the data: c1


IndexError: index 83 is out of bounds for axis 0 with size 2

IndexError: index 99 is out of bounds for axis 0 with size 2

IndexError: index 30 is out of bounds for axis 0 with size 2